# Algorithm customization

`py4dgeo` does not only provide a high performance implementation of the M3C2 base algorithm and some of it variants. It also allows you to rapidly prototype new algorithms in Python. We will demonstrate the necessary concepts by implementing some dummy algorithms without geographic relevance. First, we do the necessary setup, please consult the [M3C2 notebook](m3c2.ipynb) for details.

**Implemented by**  
Dominic Kempf ([@dokempf](https://github.com/dokempf), Heidelberg University)


In [ ]:
import numpy as np
import py4dgeo
import pooch

In order to test our dummy algorithms in this notebook, we load some point clouds:

In [ ]:
# Set up pooch to download data from Zenodo
p = pooch.Pooch(base_url="doi:10.5281/zenodo.16751963/", path=pooch.os_cache("py4dgeo"))
p.load_registry_from_doi()

try:
    # Download and extract the dataset
    p.fetch("usage_data.zip", processor=pooch.Unzip(members=["usage_data"]))

    # Define path to the extracted data
    data_path = p.path / "usage_data.zip.unzip" / "usage_data"
    print(f"Data path: {data_path}")

    # Read XYZ files from the extracted directory
    epoch1, epoch2 = py4dgeo.read_from_xyz(
        data_path / "plane_horizontal_t1.xyz", data_path / "plane_horizontal_t2.xyz"
    )

except Exception as e:
    print(f"Failed to download or extract data: {e}")

In [ ]:
corepoints = epoch1.cloud[::100]

## Inherting from the algorithm class

Each algorithm is represented by a class that inherits from `M3C2LikeAlgorithm`. It does not need to inherit directly from that class, but can e.g. also inherit from a more specialized class like `M3C2`. Our first algorithm will behave exactly like `M3C2` only that it reports a different name:

In [ ]:
class RenamedAlgorithm(py4dgeo.M3C2):
    @property
    def name(self):
        return "My super-duper M3C2 algorithm"

In the following, we will go over possible customization points for derived algorithm classes.

## Changing search directions

Next, we switch to another method of determining the search direction, namely the constant direction `(0, 0, 1)`:

In [ ]:
class DirectionAlgorithm(RenamedAlgorithm):
    def directions(self):
        return np.array([0, 0, 1])

In [ ]:
DirectionAlgorithm(epochs=(epoch1, epoch2), corepoints=corepoints, cyl_radius=5.0).run()

In the above, we chose a constant vector across all corepoints by providing an array of shape `(1x3)`. Alternatively we may provide an array of the same shape as the corepoints array to implement a normal direction that varies for each core point.

## Adding Python callbacks to the C++ implementation

`py4dgeo` implements the M3C2 algorithm in performance-oriented C++. The implementation is substructured as follows: The main algorithm for distance calculation gets passed several callback functions that it calls during distance calculation. For each of these callback functions, there are two implementations:

* An efficient C++ function that is exposed through Python bindings
* A pure Python function that serves as a fallback/reference implementation.

In order to customize the algorithm behaviour, you can also provide your own implementation (either in Python or in C++). See the following example where we like this:

In [ ]:
def my_custom_workingset_finder(*args):
    print("I was called and returned a single point")
    return np.zeros((1, 3))

In [ ]:
class CallbackAlgorithm(RenamedAlgorithm):
    def callback_workingset_finder(self):
        return my_custom_workingset_finder

In [ ]:
CallbackAlgorithm(
    epochs=(epoch1, epoch2),
    corepoints=corepoints,
    cyl_radius=2.0,
    normal_radii=(1.0, 2.0),
).run()

In order to learn about what possible callbacks there are and what arguments they are expecting, please have a look at the Python fallback implementations in [fallback.py](https://github.com/3dgeo-heidelberg/py4dgeo/blob/main/py4dgeo/fallback.py) *Note: There will be better documentation about this in the future!* For educational, testing and debugging purposes, there is an implementation of the M3C2 base algorithm that exclusively uses Python fallbacks:

In [ ]:
from py4dgeo.fallback import PythonFallbackM3C2

In [ ]:
PythonFallbackM3C2(
    epochs=(epoch1, epoch2),
    corepoints=corepoints,
    cyl_radius=2.0,
    normal_radii=(1.0, 2.0),
).run()

<div class="alert alert-block alert-warning"><b>Important:</b> Using Python callbacks does considerably slow down your algorithm. While this is true for sequential runs, the effects are even more substantial when applying multi-threading. In the worst case (where you spend all your runtime in Python callbacks), your parallel performance will degrade to sequential. Use this feature for prototyping, but provide a C++ implementation of your callback for production runs.</div>

## Other customization

If your algorithm requires a different customization point, please open an issue on [the py4dgeo issue tracker](https://github.com/3dgeo-heidelberg/py4dgeo/issues).